$r_i - r_f = \alpha_i + \beta_i(r_M - r_f) + s_iSMB + h_iHML + \epsilon_i$

$r_i$: returns of asset i

$r_f$: risk free rate

$r_i - r_f$: Excess retuen of asset i

$\alpha_i + \beta_i(r_M - r_f)$: CPAM Factor

$s_iSMB$: capture size effect

$h_iHML$: capture value effect


In [10]:
import datetime as dt
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import statsmodels.api as sm

In [20]:
end = dt.date(2020, 6, 30)
start = dt.date(end.year - 5, end.month, end.day)


In [21]:
tickers = ["FDGRX"]
stocks = pdr.get_data_yahoo(tickers, start, end)
stocks.head()

Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,FDGRX,FDGRX,FDGRX,FDGRX,FDGRX,FDGRX
Date,,,,,,
2015-06-30,10.043259,13.987,13.987,13.987,13.987,0.0
2015-07-01,10.074853,14.031,14.031,14.031,14.031,0.0
2015-07-02,10.062646,14.014,14.014,14.014,14.014,0.0
2015-07-06,10.036797,13.978,13.978,13.978,13.978,0.0
2015-07-07,10.072698,14.028,14.028,14.028,14.028,0.0


In [25]:
stocks_price = stocks["Adj Close"]
stocks_price.head()

Symbols,FDGRX
Date,
2015-06-30,10.043259
2015-07-01,10.074853
2015-07-02,10.062646
2015-07-06,10.036797
2015-07-07,10.072698


In [32]:
returns = stocks_price.pct_change().dropna()
returns.head()

Symbols,FDGRX
Date,
2015-07-01,0.003146
2015-07-02,-0.001212
2015-07-06,-0.002569
2015-07-07,0.003577
2015-07-08,-0.020031


In [33]:
returns_monthly = returns.resample("M").agg(lambda x: (x + 1).prod() - 1)
returns_monthly

Symbols,FDGRX
Date,
2015-07-31,0.037106
2015-08-31,-0.067351
2015-09-30,-0.040284
2015-10-31,0.083025
2015-11-30,0.020907
2015-12-31,-0.010148
2016-01-31,-0.106114
2016-02-29,-0.012863
2016-03-31,0.068304


$r_i - r_f = \alpha_i + \beta_i(r_M - r_f) + s_iSMB + h_iHML + \epsilon_i$


In [35]:
factors = pdr.DataReader("F-F_Research_Data_Factors", "famafrench", start, end)[0]
factors

,Mkt-RF,SMB,HML,RF
Date,,,,
2015-06,-1.53,2.90,-0.78,0.00
2015-07,1.54,-4.20,-4.07,0.00
2015-08,-6.04,0.36,2.80,0.00
2015-09,-3.07,-2.63,0.58,0.00
2015-10,7.75,-1.86,-0.45,0.00
...,...,...,...,...
2020-02,-8.13,1.03,-3.79,0.12
2020-03,-13.38,-4.89,-14.02,0.12
2020-04,13.65,2.47,-1.18,0.00


In [39]:
factors = factors.drop(index="2015-06")
factors

,Mkt-RF,SMB,HML,RF
Date,,,,
2015-07,1.54,-4.20,-4.07,0.00
2015-08,-6.04,0.36,2.80,0.00
2015-09,-3.07,-2.63,0.58,0.00
2015-10,7.75,-1.86,-0.45,0.00
2015-11,0.56,3.60,-0.39,0.00
2015-12,-2.17,-2.81,-2.59,0.01
2016-01,-5.77,-3.39,2.07,0.01
2016-02,-0.08,0.81,-0.58,0.02
2016-03,6.96,0.75,1.10,0.02


In [40]:
factors.info()

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 60 entries, 2015-07 to 2020-06
Freq: M
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Mkt-RF  60 non-null     float64
 1   SMB     60 non-null     float64
 2   HML     60 non-null     float64
 3   RF      60 non-null     float64
dtypes: float64(4)
memory usage: 2.3 KB


In [41]:
returns_monthly.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 60 entries, 2015-07-31 to 2020-06-30
Freq: M
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   FDGRX   60 non-null     float64
dtypes: float64(1)
memory usage: 960.0 bytes


In [47]:
returns_monthly.index = factors.index
returns_monthly.head()

Symbols,FDGRX
Date,
2015-07,0.037106
2015-08,-0.067351
2015-09,-0.040284
2015-10,0.083025
2015-11,0.020907


In [48]:
df = pd.merge(left=returns_monthly,
              right=factors,
              on="Date")
df

,FDGRX,Mkt-RF,SMB,HML,RF
Date,,,,,
2015-07,0.037106,1.54,-4.20,-4.07,0.00
2015-08,-0.067351,-6.04,0.36,2.80,0.00
2015-09,-0.040284,-3.07,-2.63,0.58,0.00
2015-10,0.083025,7.75,-1.86,-0.45,0.00
2015-11,0.020907,0.56,3.60,-0.39,0.00
2015-12,-0.010148,-2.17,-2.81,-2.59,0.01
2016-01,-0.106114,-5.77,-3.39,2.07,0.01
2016-02,-0.012863,-0.08,0.81,-0.58,0.02
2016-03,0.068304,6.96,0.75,1.10,0.02


In [49]:
df[["Mkt-RF", "SMB", "HML", "RF"]] = df[["Mkt-RF", "SMB", "HML", "RF"]] / 100.0

In [50]:
df

,FDGRX,Mkt-RF,SMB,HML,RF
Date,,,,,
2015-07,0.037106,0.0154,-0.0420,-0.0407,0.0000
2015-08,-0.067351,-0.0604,0.0036,0.0280,0.0000
2015-09,-0.040284,-0.0307,-0.0263,0.0058,0.0000
2015-10,0.083025,0.0775,-0.0186,-0.0045,0.0000
2015-11,0.020907,0.0056,0.0360,-0.0039,0.0000
2015-12,-0.010148,-0.0217,-0.0281,-0.0259,0.0001
2016-01,-0.106114,-0.0577,-0.0339,0.0207,0.0001
2016-02,-0.012863,-0.0008,0.0081,-0.0058,0.0002
2016-03,0.068304,0.0696,0.0075,0.0110,0.0002


$r_i - r_f = \alpha_i + \beta_i(r_M - r_f) + s_iSMB + h_iHML + \epsilon_i$


In [51]:
df["Excess Retuen"] = df["FDGRX"] - df["RF"]
df

,FDGRX,Mkt-RF,SMB,HML,RF,Excess Retuen
Date,,,,,,
2015-07,0.037106,0.0154,-0.0420,-0.0407,0.0000,0.037106
2015-08,-0.067351,-0.0604,0.0036,0.0280,0.0000,-0.067351
2015-09,-0.040284,-0.0307,-0.0263,0.0058,0.0000,-0.040284
2015-10,0.083025,0.0775,-0.0186,-0.0045,0.0000,0.083025
2015-11,0.020907,0.0056,0.0360,-0.0039,0.0000,0.020907
2015-12,-0.010148,-0.0217,-0.0281,-0.0259,0.0001,-0.010248
2016-01,-0.106114,-0.0577,-0.0339,0.0207,0.0001,-0.106214
2016-02,-0.012863,-0.0008,0.0081,-0.0058,0.0002,-0.013063
2016-03,0.068304,0.0696,0.0075,0.0110,0.0002,0.068104


In [52]:
y = df["Excess Retuen"]
X = df[["Mkt-RF", "SMB", "HML"]]
X_sm = sm.add_constant(X)

In [54]:
model = sm.OLS(y, X_sm)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:          Excess Retuen   R-squared:                       0.936
Model:                            OLS   Adj. R-squared:                  0.933
Method:                 Least Squares   F-statistic:                     273.7
Date:                Sun, 31 Oct 2021   Prob (F-statistic):           2.05e-33
Time:                        10:01:18   Log-Likelihood:                 172.21
No. Observations:                  60   AIC:                            -336.4
Df Residuals:                      56   BIC:                            -328.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0020      0.002      1.006      0.3